In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
from itertools import islice

In [2]:
# Create DataFrame from CSV
t0 = time.time()
filtered_cooccurring_df = pd.read_csv('../../Data/filtered_cooccurring_TF.csv')
filtered_cooccurring_df.index = filtered_cooccurring_df.index - 1
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_cooccurring_df.head(3)

Run time: 42.38177013397217 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
-1,NaN,NaN,NaN,53468.0,NaN,4528.6433,5362.4272,42651.0,5517.0,665.7547,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,240.0
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_cooccurring_df[filtered_cooccurring_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [5]:
# Create a new DataFrame with co-occurring word columns
new_lyric_TF_df = filtered_cooccurring_df.drop([-1])
new_lyric_TF_df = new_lyric_TF_df.drop(columns=new_lyric_TF_df.columns[17:14144])
new_lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# Create a list of the overall top ten words 
word_count_df = filtered_cooccurring_df.drop(columns=filtered_cooccurring_df.columns[:14144])#[:1]
word_count_df = word_count_df.sort_values(by=[-1], axis=1, ascending=False)

total_count = len(word_count_df) - 1
top_words = list(word_count_df.columns[0:10])
word_count_df = word_count_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(word_count_df[word_count_df[word]!=0])-1) / total_count * 100
    top_ten[word] = round(percent, 1)

top_word_counts = list(word_count_df.loc[-1])[0:10]
index = 0
for word in top_ten:
    print(word)
    print('Frequency & percent: (', top_word_counts[index], ',', top_ten[word], ')')
    index+=1

oh oh
Frequency & percent: ( 4588.0 , 8.3 )
yeah yeah
Frequency & percent: ( 3169.0 , 8.9 )
ooh ooh
Frequency & percent: ( 2568.0 , 4.0 )
na na
Frequency & percent: ( 2425.0 , 0.9 )
dont know
Frequency & percent: ( 2363.0 , 12.8 )
la la
Frequency & percent: ( 2323.0 , 1.1 )
dont wanna
Frequency & percent: ( 1487.0 , 6.1 )
let go
Frequency & percent: ( 1215.0 , 5.8 )
im gonna
Frequency & percent: ( 1200.0 , 6.0 )
oh yeah
Frequency & percent: ( 1018.0 , 5.7 )


In [11]:
# Create a dictionary of the top ten co-occurring words for each category
t0 = time.time()
df_rows = []
blues_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='blues'].copy()
count_blues = len(blues_df)
blues_df.loc[0] = blues_df.sum(numeric_only=True)
blues_words = blues_df.sort_index()[blues_df.columns[17:]][0:1]
blues_words = blues_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(blues_words.columns[0:10])
blues_df = blues_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(blues_df[blues_df[word]!=0])-1) / count_blues * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(blues_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

classical_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='classical'].copy()
count_classical = len(classical_df)
classical_df.loc[0] = classical_df.sum(numeric_only=True)
classical_words = classical_df.sort_index()[classical_df.columns[18:]][0:1]
classical_words = classical_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(classical_words.columns[0:10])
classical_df = classical_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(classical_df[classical_df[word]!=0])-1) / count_classical * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(classical_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

country_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='country'].copy()
count_country = len(country_df)
country_df.loc[0] = country_df.sum(numeric_only=True)
country_words = country_df.sort_index()[country_df.columns[18:]][0:1]
country_words = country_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(country_words.columns[0:10])
country_df = country_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(country_df[country_df[word]!=0])-1) / count_country * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(country_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

funk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='funk'].copy()
count_funk = len(funk_df)
funk_df.loc[0] = funk_df.sum(numeric_only=True)
funk_words = funk_df.sort_index()[funk_df.columns[18:]][0:1]
funk_words = funk_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(funk_words.columns[0:10])
funk_df = funk_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(funk_df[funk_df[word]!=0])-1) / count_funk * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(funk_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

hiphop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='hiphop'].copy()
count_hiphop = len(hiphop_df)
hiphop_df.loc[0] = hiphop_df.sum(numeric_only=True)
hiphop_words = hiphop_df.sort_index()[hiphop_df.columns[18:]][0:1]
hiphop_words = hiphop_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(hiphop_words.columns[0:10])
hiphop_df = hiphop_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(hiphop_df[hiphop_df[word]!=0])-1) / count_hiphop * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(hiphop_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

indie_alt_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='indie_alt'].copy()
count_indie_alt = len(indie_alt_df)
indie_alt_df.loc[0] = indie_alt_df.sum(numeric_only=True)
indie_alt_words = indie_alt_df.sort_index()[indie_alt_df.columns[18:]][0:1]
indie_alt_words = indie_alt_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(indie_alt_words.columns[0:10])
indie_alt_df = indie_alt_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(indie_alt_df[indie_alt_df[word]!=0])-1) / count_indie_alt * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(indie_alt_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

jazz_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='jazz'].copy()
count_jazz = len(jazz_df)
jazz_df.loc[0] = jazz_df.sum(numeric_only=True)
jazz_words = jazz_df.sort_index()[jazz_df.columns[18:]][0:1]
jazz_words = jazz_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(jazz_words.columns[0:10])
jazz_df = jazz_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(jazz_df[jazz_df[word]!=0])-1) / count_jazz * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(jazz_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

metal_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='metal'].copy()
count_metal = len(metal_df)
metal_df.loc[0] = metal_df.sum(numeric_only=True)
metal_words = metal_df.sort_index()[metal_df.columns[18:]][0:1]
metal_words = metal_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(metal_words.columns[0:10])
metal_df = metal_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(metal_df[metal_df[word]!=0])-1) / count_metal * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(metal_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

pop_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='pop'].copy()
count_pop = len(pop_df)
pop_df.loc[-1] = pop_df.sum(numeric_only=True)
pop_words = pop_df.sort_index()[pop_df.columns[18:]][:1]
pop_words = pop_words.sort_values(by=[-1], axis=1, ascending=False)
top_words = list(pop_words.columns[0:10])
pop_df = pop_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(pop_df[pop_df[word]!=0])-1) / count_pop * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(pop_words.loc[-1])[0:10])
df_rows.append(list(top_ten.values()))

punk_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='punk'].copy()
count_punk = len(punk_df)
punk_df.loc[0] = punk_df.sum(numeric_only=True)
punk_words = punk_df.sort_index()[punk_df.columns[18:]][0:1]
punk_words = punk_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(punk_words.columns[0:10])
punk_df = punk_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(punk_df[punk_df[word]!=0])-1) / count_punk * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(punk_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

rnb_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rnb'].copy()
count_rnb = len(rnb_df)
rnb_df.loc[0] = rnb_df.sum(numeric_only=True)
rnb_words = rnb_df.sort_index()[rnb_df.columns[18:]][0:1]
rnb_words = rnb_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(rnb_words.columns[0:10])
rnb_df = rnb_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(rnb_df[rnb_df[word]!=0])-1) / count_rnb * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(rnb_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

rock_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='rock'].copy()
count_rock = len(rock_df)
rock_df.loc[0] = rock_df.sum(numeric_only=True)
rock_words = rock_df.sort_index()[rock_df.columns[18:]][0:1]
rock_words = rock_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(rock_words.columns[0:10])
rock_df = rock_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(rock_df[rock_df[word]!=0])-1) / count_rock * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(rock_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

romance_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='romance'].copy()
count_romance = len(romance_df)
romance_df.loc[0] = romance_df.sum(numeric_only=True)
romance_words = romance_df.sort_index()[romance_df.columns[18:]][0:1]
romance_words = romance_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(romance_words.columns[0:10])
romance_df = romance_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(romance_df[romance_df[word]!=0])-1) / count_romance * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(romance_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

soul_df = new_lyric_TF_df[new_lyric_TF_df['category_name']=='soul'].copy()
count_soul = len(soul_df)
soul_df.loc[0] = soul_df.sum(numeric_only=True)
soul_words = soul_df.sort_index()[soul_df.columns[18:]][0:1]
soul_words = soul_words.sort_values(by=[0], axis=1, ascending=False)
top_words = list(soul_words.columns[0:10])
soul_df = soul_df[top_words]
top_ten = {}
for word in top_words:
    percent = (len(soul_df[soul_df[word]!=0])-1) / count_soul * 100
    top_ten[word] = round(percent, 1)
df_rows.append(top_words)
df_rows.append(list(soul_words.loc[0])[0:10])
df_rows.append(list(top_ten.values()))

t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 0.6392710208892822 seconds


In [12]:
# Create dictionary of top word frequencies
unique_top_words = []
for i in range(len(df_rows)):
    if i%3 == 0:
        unique_top_words.extend(df_rows[i])
unique_top_words = list(set(unique_top_words))

top_word_freq = dict.fromkeys(unique_top_words, 0)
for i in range(len(df_rows)):
    if i%3 == 0:
        for word in df_rows[i]:
            top_word_freq[word]+=1
print(len(top_word_freq))
top_word_freq

46


{'ive got': 5,
 'doo doo': 3,
 'let go': 8,
 'yeah yeah': 12,
 'da da': 5,
 'uh huh': 1,
 'dont want': 3,
 'baby baby': 2,
 'know im': 1,
 'yeah !': 1,
 'youre gonna': 1,
 'cause im': 2,
 'du du': 2,
 'ooh ooh': 10,
 'uh uh': 1,
 'gotta get': 1,
 'im gonna': 5,
 'good time': 1,
 'thats way': 1,
 'round round': 1,
 'ill never': 1,
 'love love': 3,
 'dont know': 13,
 'dont wanna': 8,
 'nah nah': 1,
 'ya ya': 1,
 'im going': 1,
 'dont let': 1,
 'little bit': 1,
 'baby dont': 1,
 'na na': 8,
 'oh baby': 1,
 'hey hey': 4,
 'oh yeah': 4,
 'go go': 1,
 'cant see': 1,
 'aint got': 1,
 'la la': 10,
 'come come': 1,
 'time time': 1,
 'know know': 1,
 'ah ah': 4,
 'cant get': 1,
 'dont stop': 1,
 'feel like': 2,
 'get get': 2}

In [14]:
# Print top words that are unique to one genre
unique_to_one = dict(filter(lambda key: key[1] == 1, top_word_freq.items()))
print(len(unique_to_one))
unique_to_one

25


{'uh huh': 1,
 'know im': 1,
 'yeah !': 1,
 'youre gonna': 1,
 'uh uh': 1,
 'gotta get': 1,
 'good time': 1,
 'thats way': 1,
 'round round': 1,
 'ill never': 1,
 'nah nah': 1,
 'ya ya': 1,
 'im going': 1,
 'dont let': 1,
 'little bit': 1,
 'baby dont': 1,
 'oh baby': 1,
 'go go': 1,
 'cant see': 1,
 'aint got': 1,
 'come come': 1,
 'time time': 1,
 'know know': 1,
 'cant get': 1,
 'dont stop': 1}

In [51]:
# Print top words that are unique to two genre
unique_to_two = dict(filter(lambda key: key[1] == 2, top_word_freq.items()))
print(len(unique_to_two))
unique_to_two

2


{'du du': 2, 'get get': 2}

In [15]:
# Create DataFrame of top ten words
categories = np.array(['blues', 'blues', 'blues', 'classical', 'classical', 'classical', 'country', 'country', 'country', 
                       'funk', 'funk', 'funk', 'hiphop', 'hiphop', 'hiphop', 'indie_alt', 'indie_alt', 'indie_alt', 
                       'jazz', 'jazz', 'jazz', 'metal', 'metal', 'metal', 'pop', 'pop', 'pop', 'punk', 'punk', 'punk', 
                       'rnb', 'rnb', 'rnb', 'rock', 'rock', 'rock', 'romance', 'romance', 'romance', 'soul', 'soul', 'soul'])
arrays = [categories, np.array(['words', 'frequencies', 'percents', 'words', 'frequencies', 'percents', 'words', 
                       'frequencies', 'percents', 'words', 'frequencies', 'percents', 'words', 'frequencies', 
                       'percents', 'words', 'frequencies', 'percents', 'words', 'frequencies', 'percents', 'words',
                       'frequencies', 'percents', 'words', 'frequencies', 'percents', 'words', 'frequencies', 
                       'percents', 'words', 'frequencies', 'percents', 'words', 'frequencies', 'percents', 'words', 
                       'frequencies', 'percents', 'words', 'frequencies', 'percents'])]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples)
top_cooccurring_df = pd.DataFrame(df_rows, index=arrays, columns=range(1,11))
top_cooccurring_df

1          2            3           4   \
blues     words        dont know   im going      hey hey    im gonna   
          frequencies        122         99           93          88   
          percents            13        8.3          3.1        12.7   
classical words            da da      ah ah        du du  dont wanna   
          frequencies         26         24           15          13   
          percents           1.1        1.1          1.1         3.2   
country   words        dont know      na na   dont wanna     ooh ooh   
          frequencies        371        352          270         267   
          percents          14.5        0.7          5.3         2.5   
funk      words          get get  dont stop      hey hey   yeah yeah   
          frequencies        251        200          199         116   
          percents             6        5.3          7.8        13.1   
hiphop    words        yeah yeah      na na    dont know     know im   
          frequencies        764        320          260         214   
          percents          25.5        0.9         20.8          19   
indie_alt words          ooh ooh      la la      doo doo   dont know   
          frequencies        283        214          169         150   
          percents             5        1.2          0.5         9.8   
jazz      words            na na  thats way  round round      let go   
          frequencies         66         61           53          39   
          percents           0.5        1.5            1           3   
metal     words          hey hey  dont know        da da      let go   
          frequencies        119        119          103          99   
          percents           0.4        5.7          0.1         4.6   
pop       words          ooh ooh      la la        na na   dont know   
          frequencies        793        636          554         303   
          percents           8.2        2.7          1.7        16.9   
punk      words            la la  yeah yeah    dont know       na na   
          frequencies        164        158          148         140   
          percents           0.7        3.9         11.6         1.1   
rnb       words        yeah yeah  love love      ooh ooh   dont know   
          frequencies        580        282          249         205   
          percents          24.9        9.1           11        18.5   
rock      words            na na  yeah yeah      doo doo   dont know   
          frequencies        687        432          428         339   
          percents           0.9          7          0.4        11.9   
romance   words            la la  dont know    yeah yeah   time time   
          frequencies        118        102           85          73   
          percents           2.5       12.5          8.9         3.6   
soul      words            la la  yeah yeah      ooh ooh       du du   
          frequencies        358        275          235         202   
          percents           1.5       17.3         10.9         0.5   

                               5          6           7           8   \
blues     words         dont want    ooh ooh     ive got     nah nah   
          frequencies          66         62          60          58   
          percents            5.7        2.8         7.5         0.2   
classical words         dont know      la la   gotta get   come come   
          frequencies          12         10          10           8   
          percents            7.4        2.1         1.1         2.1   
country   words          im gonna  yeah yeah     ive got       la la   
          frequencies         230        188         137         137   
          percents            7.4        5.6         5.6         0.4   
funk      words           ooh ooh      na na     oh yeah    im gonna   
          frequencies          86         77          77          69   
          percents            5.3        2.1         9.9          11 

In [23]:
# Save DataFrame to CSV
top_cooccurring_df.to_csv('../../Data/Analysis/top_cooccurring_words.csv')

# Conclusions about top co-occurring words
- From the total 140 top ten co-occurring words from each of the 14 genres, 46 were unique
- 25 of the top ten co-occurring words are unique to one genre
- 2 of the top ten co-occurring words are unique to two genres
- 'dont know' is in the top ten co-occurring words for 13 out of 14 genres